### Imports

In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import constants, utils

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = utils.get_openai_api_key()

### Built in Function

In [4]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import ChatOpenAI

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatOpenAI(temperature=0, model_name=constants.MODEL_NAME)
chain = load_summarize_chain(llm, chain_type="stuff")

result = chain.invoke(docs)

print(result["output_text"])

The article discusses the concept of building autonomous agents powered by Large Language Models (LLMs). It covers components such as planning, memory, and tool use, with examples like AutoGPT and GPT-Engineer. Challenges include finite context length, reliability of natural language interface, and long-term planning difficulties. The article also references various studies and projects related to LLM-powered agents.


### Option 1: Stuff or Simple Prompt

In [7]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate

# Define prompt
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm = ChatOpenAI(temperature=0, model_name=constants.MODEL_NAME)
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

docs = loader.load()
print(stuff_chain.invoke(docs)["output_text"])

/home/pedro/Documents/Virtual-Environments/ai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The article discusses the concept of building autonomous agents powered by Large Language Models (LLMs). It covers components such as planning, memory, and tool use, with examples like AutoGPT and GPT-Engineer. Challenges include finite context length, planning difficulties, and reliability of natural language interfaces. The article also references various studies and projects related to LLM-powered agents.


### Option 2: Map-Reduce

Map

In [8]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

llm = ChatOpenAI(temperature=0, model_name=constants.MODEL_NAME)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

Reduce

In [9]:
reduce_template = """The following is set of summaries:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_prompt

PromptTemplate(input_variables=['docs'], template='The following is set of summaries:\n{docs}\nTake these and distill it into a final, consolidated summary of the main themes. \nHelpful Answer:')

In [10]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [11]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [12]:
result = map_reduce_chain.invoke(split_docs)

print(result["output_text"])

The main themes identified in the set of documents revolve around the utilization of Large Language Models (LLMs) in autonomous agents for problem-solving and general tasks. Key components include planning, memory, tool use, and integration of reasoning and acting. Challenges in building and utilizing LLM-powered agents are addressed, along with frameworks for self-reflection and incremental improvement. The documents also discuss methods for improving model outputs, contextual learning, and performance comparison with baselines. Additionally, topics such as memory optimization, similarity search algorithms, tool augmentation, and task planning are explored. Case studies and proof-of-concept examples showcase the capabilities of LLM-powered agents in various applications, including scientific discovery, generative agents simulation, and task delegation. Challenges in real-world applications, performance evaluation, and case studies in drug discovery and synthesis are also highlighted. 

### Option 3: Refine

In [13]:
chain = load_summarize_chain(llm, chain_type="refine")
result = chain.invoke(split_docs)

print(result["output_text"])

{
    "thoughts": {
        "text": "The additional context provided introduces the challenges in long-term planning, task decomposition, and the reliability of natural language interfaces in LLM-powered autonomous agents.",
        "reasoning": "To refine the original summary, we will need to address the challenges faced by LLMs in adjusting plans, exploring solution spaces, and ensuring the reliability of natural language interfaces in the code implementation. The new references highlight the advancements and applications of LLMs in autonomous agents.",
        "plan": "- We will incorporate mechanisms to address the challenges in long-term planning and task decomposition, ensuring robustness and adaptability in the code. Additionally, we will focus on improving the reliability of natural language interfaces by handling formatting errors and rebellious behavior. We will also explore the latest research and developments in LLM-powered autonomous agents.",
        "criticism": "N/A",
 